<font size = 40 color=darkgreen>Dutch Rental Prices</font><br>
*Both my friends and myself have often asked the question, 'How much rent could I charge for my apartment?' The question becomes very complex as we start assuming that location, size, amenities, furnished, and many more attributes need to be considered when deciding on a rental price. My goal for this project is to build a model that will weigh all the attributes available using data from a popular rental website in the Netherlands, Kamernet. <BR>  
*Creating a regression model to predict the rental prices in the Netherlands using data from kamernet.nl*<br>
**Steps:**<br>
1. Importing libraries and reading data
2. EDA and re-shaping data for ML pre-processing 
3. PyCaret setup
4. Adjust base data, repeat compare and create steps

# <font color=teal>Import data</font>

In [2]:
# Data manipulation
import pandas as pd
import numpy as np

# ML libraries
import pycaret.regression as py
from pycaret.regression import *

# Options for pandas
pd.options.display.max_columns
pd.options.display.max_rows = 30

# Visualizations
from matplotlib import pyplot as plt
import seaborn as sns
import missingno as msno
import chart_studio.plotly as pl
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode

In [3]:
# Reading json data from Kamernet

#Git
source = 'https://github.com/michael-william/Netherlands-Rental-Prices/raw/master/properties-trim.json'
df=pd.read_json(source, lines=True)


# <font color=teal>EDA and reshaping data</font>

> **<font color=brown>Overview</font>**
> 1. Dropping non-needed columns
> 2. Missing values
> 3. Cardinality of categorical values

In [27]:
# Creating copy of main df
data = df[['areaSqm', 'furnish', 'latitude', 'longitude', 'propertyType', 'roommates','rent']]

In [28]:
# converting 'roommates' feature to 'int' and creating a new binary feature called 'shared'
data['roommates'] = data.roommates.replace(to_replace='None', value=0)
data['roommates'] = data.roommates.replace(to_replace='More than 8', value=9)
data['roommates'] = data.roommates.replace(to_replace='Unknown', value=0)
data['roommates'] = data.roommates.replace(to_replace='nan', value=0)
data['roommates'] = data.roommates.fillna(0)
data['roommates'] = data.roommates.astype('int')
data['shared'] = [1 if x>0 else 0 for x in data.roommates]

In [29]:
data['furnish'] = ['Furnished' if x=='Furnished' else 'Unfurnished' for x in data.furnish]

In [30]:
data.to_csv("ml_data.csv")

In [31]:
py.setup(data, target='rent', silent=True)

 
Setup Succesfully Completed!


(       areaSqm   latitude  longitude  furnish_Furnished  \
 0         14.0  51.896601   4.514993                0.0   
 1         30.0  52.370200   4.920721                1.0   
 2         11.0  52.350880   4.854786                1.0   
 3         16.0  53.013494   6.561012                0.0   
 4         22.0  51.932871   4.479732                0.0   
 ...        ...        ...        ...                ...   
 12825     35.0  52.368030   5.205308                0.0   
 12826     10.0  51.690365   5.311952                1.0   
 12827     12.0  52.010965   4.336536                0.0   
 12828     16.0  52.008520   4.390628                0.0   
 12829     21.0  51.906095   4.445226                0.0   
 
        propertyType_Anti-squat  propertyType_Apartment  propertyType_Room  \
 0                          0.0                     0.0                1.0   
 1                          0.0                     0.0                0.0   
 2                          0.0             

In [37]:
rf_new = py.create_model('rf', fold=5)

MAE         MSE      RMSE      R2   RMSLE    MAPE
0     105.4211  33519.9576  183.0846  0.8317  0.2773  0.4887
1     104.5692  36502.5423  191.0564  0.8212  0.3458  1.2248
2     110.8301  33476.8500  182.9668  0.8424  0.3220  0.9078
3     109.3855  37382.9896  193.3468  0.8101  0.2678  0.3758
4     105.8124  34210.3304  184.9603  0.8266  0.3682  1.2428
Mean  107.2037  35018.5340  187.0830  0.8264  0.3162  0.8480
SD      2.4481   1616.7031    4.3002  0.0107  0.0386  0.3615

In [38]:
rf = finalize_model(rf_new)

In [39]:
label_df = predict_model('dutch_pycaret_rf', data=data)

In [40]:
label_df

areaSqm      furnish   latitude  longitude       propertyType  \
0           14  Unfurnished  51.896601   4.514993               Room   
1           30    Furnished  52.370200   4.920721             Studio   
2           11    Furnished  52.350880   4.854786               Room   
3           16  Unfurnished  53.013494   6.561012               Room   
4           22  Unfurnished  51.932871   4.479732               Room   
...        ...          ...        ...        ...                ...   
12825       35  Unfurnished  52.368030   5.205308               Room   
12826       10    Furnished  51.690365   5.311952               Room   
12827       12  Unfurnished  52.010965   4.336536  Student residence   
12828       16  Unfurnished  52.008520   4.390628               Room   
12829       21  Unfurnished  51.906095   4.445226               Room   

       roommates  rent  shared      Label  
0              5   500       1   474.5200  
1              0   950       0  1052.8667  
2              1  1000       1   905.5000  
3              4   290       1   305.4433  
4              1   475       1  1300.7800  
...          ...   ...     ...        ...  
12825          1   750       1   705.1400  
12826          1   600       1   498.0000  
12827          4   395       1   416.8600  
12828          2   425       1   447.3083  
12829          2   695       1   768.0700  

[12830 rows x 9 columns]

In [41]:
prediction = predict_model('dutch_pycaret_rf', data=data)['Label']

In [43]:
prediction

0         474.5200
1        1052.8667
2         905.5000
3         305.4433
4        1300.7800
           ...    
12825     705.1400
12826     498.0000
12827     416.8600
12828     447.3083
12829     768.0700
Name: Label, Length: 12830, dtype: float64